<a href="https://colab.research.google.com/github/fabian-castano/timetabling_upb/blob/transition-to-gurobi/ModeloWilliam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv("Facultad_ingenieria.csv")
df.columns
df["ASIGNATURA "].fillna(method="ffill",inplace=True)
df.groupby("ASIGNATURA ").sum().to_clipboard()
df_melt=df.melt(id_vars=["ASIGNATURA "],var_name="Docente")
df_melt=df_melt[(df_melt["value"]>0) & (df_melt.Docente!="Unnamed: 24")]

In [ ]:
set(df_melt['ASIGNATURA '])
df_melt["code"]=df_melt["ASIGNATURA "].apply(lambda x: x[:9])

# Creamos un diccionario con la relación código asignatura - nombre asignatura
dict_courses=df_melt.set_index("code").to_dict()["ASIGNATURA "]


In [ ]:
oferta_x_curso={i:np.random.choice([1,2],p=[0.9,0.1]) for i in set(df_melt["code"])}

In [ ]:
L={str(i):[str(i)+"-"+str(j)  for j in range(1,oferta_x_curso[i]+1)] for i in dict_courses.keys()}
grupos=[]
for l in L.values():
    for ele in l:
        grupos.append(ele)

curso_base={str(i)+"-"+str(j):str(i)  for i in dict_courses.keys() for j in range(1,oferta_x_curso[i]+1) }


In [ ]:
horas_grupo={i:3 for i in grupos}
max_horas_prof={i:np.random.choice([8,16,20],p=[0.25,0.5,0.25]) for i in set(df_melt["Docente"])}

In [ ]:
dict_profs=df_melt.groupby("Docente").agg(list)[["code"]].reset_index()
dict_profs=dict_profs.set_index("Docente").to_dict()["code"]


In [ ]:
df_melt["id-tuple"]=df_melt.apply(lambda x: (x["Docente"],x["code"]),axis=1)
theta=df_melt.set_index("id-tuple").to_dict()["value"]


In [ ]:
num_grupos={1:1,2:3}

In [ ]:
max_cursos=6
I=set(dict_profs.keys())
J=set(grupos)   
K=set(dict_courses.keys())
C=range(0,max_cursos+1)

tuplas=[(i,j) for i in I for j in J if (i,curso_base[j]) in theta.keys()]
prof_curso=[e for e in theta.keys()  ]
# posibles combinaciones profeso y cantidad de cursos
IC=[(i,c) for i in I for c in C]

In [ ]:

import pulp as plp

# Create the 'prob' variable to contain the problem data
prob = plp.LpProblem("Modelo William", plp.LpMaximize)

# Create problem variables from dictionary
# Docente i es asignado a grupo j (binaria)
x=plp.LpVariable.dicts("x",tuplas,0,1,plp.LpBinary)

# Docente i es asignado a c cursos diferentes
N=plp.LpVariable.dicts("N",IC,0,1,plp.LpBinary)

# Un profesor es asignado a un curso
z=plp.LpVariable.dicts("z",theta.keys(),0,1,plp.LpBinary)

# e=(i,j) representa la tupla
prob+=plp.lpSum([x[(i,j)]*theta[i,curso_base[j]] for (i,j) in tuplas])

# Cada curso tiene un docente asignado
for j in J:
    prob+=plp.lpSum([x[i,j] for i in I if (i,j) in tuplas])==1

# Contar cuantos cursos diferentes da un profesor
for i in I:
    prob+=plp.lpSum([z[(i,k)] for k in K if (i,k) in theta.keys()])==plp.lpSum([l*N[i,l] for l in C])

# el profesor tiene una cantidad de cursos asignada
for i in I:
    prob+=plp.lpSum([N[i,l] for l in C])==1

# el profesor no puede dar mas cursos de los que tiene asignados
for i in I:
    for k in K:
        if (i,k) in theta.keys():
            plp.lpSum([x[(i,j)] for j in L[k] if (i,j) in tuplas])<=z[(i,k)]*len(L[k])

    
prob.solve()


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/user/opt/anaconda3/envs/apricot-env/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/xc/t5gbnt3n5sq5bkg65xw9f1g00000gn/T/39d0a813219940c5bf113dc5c90d5da2-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/xc/t5gbnt3n5sq5bkg65xw9f1g00000gn/T/39d0a813219940c5bf113dc5c90d5da2-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 101 COLUMNS
At line 1451 RHS
At line 1548 BOUNDS
At line 1917 ENDATA
Problem MODEL has 96 rows, 368 columns and 506 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 148 - 0.00 seconds
Cgl0004I processed model has 28 rows, 87 columns (87 integer (87 of which binary)) and 160 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -148
Cbc0038I Before mini branch and bound, 73 integers 

/Users/user/opt/anaconda3/envs/apricot-env/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


1

In [ ]:
for v in prob.variables():
    if v.varValue>0.5:
        print(v.name, "=", v.varValue)

N_('Alvaro_Jose_Torres_Penagos',_0) = 1.0
N_('Andres_Felipe_Hernandez_Giraldo',_0) = 1.0
N_('Andres_Felipe_Parra_Perea',_2) = 1.0
N_('Beatriz_Elena_Hernandez_Arias',_2) = 1.0
N_('Bryan_Salcedo',_0) = 1.0
N_('Claudia_Orly_Escudero_Jimenez',_1) = 1.0
N_('Diego_lennis',_0) = 1.0
N_('Edwin_Fernando_Restrepo_Salazar',_0) = 1.0
N_('Edwin_Javier_Ortega_Zuñiga',_0) = 1.0
N_('Henry_Hurtado_',_0) = 1.0
N_('Isabel_Crsitina_Quintero_',_0) = 1.0
N_('Jairo_Arboleda_Zuñiga',_0) = 1.0
N_('Jonny_Pantoja_',_0) = 1.0
N_('Marcelo_Aguilar_',_0) = 1.0
N_('Mariela_Galindo',_0) = 1.0
N_('Marino_renjifo_',_0) = 1.0
N_('Oscar_Sanchez_',_0) = 1.0
N_('Pedro_Nel_Barbosa_Garcia',_0) = 1.0
N_('Ruben_Dario_Parra_Zuleta',_0) = 1.0
N_('William_Andres_Alzate_',_2) = 1.0
N_('Yaqueline_Quiñones_Rodriguez',_0) = 1.0
N_('carlos_saldarriaga_',_0) = 1.0
N_('paola_salazar_',_0) = 1.0
x_('Alvaro_Jose_Torres_Penagos',_'8830_0004_1') = 1.0
x_('Alvaro_Jose_Torres_Penagos',_'8830_0004_2') = 1.0
x_('Alvaro_Jose_Torres_Penagos',_'883

In [ ]:
import simpy
import random

SEED = 42
MACHINES = 2     # amount of machines
INTERVAL = 10.0  # mean time between two jobs
DURATION = 5.0   # mean processing time of a job
JOBS = 100       # number of jobs that have to be completed

def production(env, name, machines):
    """ simulates job execution, i.e. production on the machine
   
    used for modelling the production process on the machine,
    once job execution on the machine has been initiated

    Args:
    # env (simpy.Environment):  contains simulation state 
    # name (str): name of job being executed
    # machines (simpy.Resource): can be used by limited number of processes 

    """
    with machines.request() as request:
        yield request
        print(f"{name} started at time {env.now}")
        yield env.timeout(DURATION)
        print(f"{name} completed at time {env.now}")

def generate_jobs(env, machines, jobs):
    """ simulates generation of new jobs

    generates new job based on interval time between jobs;
    creates job by iniviting production process

    Args:
    # env (simpy.Environment): contains simulation state 
    # machines (simpy.Resource): can be used by limited number of processes
    # jobs (int): amount of jobs to generate

    """

    for i in range(jobs):
        yield env.timeout(random.expovariate(1.0/INTERVAL))
        env.process(production(env, f"job {i+1}", machines))

# implementing actual simulation run
random.seed(SEED)
env = simpy.Environment()
machines = simpy.Resource(env, capacity=MACHINES)
env.process(generate_jobs(env, machines, JOBS))

# executing simulation run
env.run(until=1000.0)

job 1 started at time 10.200602872748009
job 2 started at time 10.453891263175398
job 1 completed at time 15.200602872748009
job 3 started at time 15.200602872748009
job 2 completed at time 15.453891263175398
job 4 started at time 16.195993760626187
job 3 completed at time 20.200602872748007
job 4 completed at time 21.195993760626187
job 5 started at time 29.531920488707016
job 5 completed at time 34.531920488707016
job 6 started at time 40.82365057669294
job 6 completed at time 45.82365057669294
job 7 started at time 63.09653157220674
job 8 started at time 64.0060556182724
job 7 completed at time 68.09653157220674
job 8 completed at time 69.0060556182724
job 9 started at time 69.4865172122373
job 10 started at time 69.78901898431089
job 9 completed at time 74.4865172122373
job 11 started at time 74.4865172122373
job 10 completed at time 74.78901898431089
job 12 started at time 79.29534122257067
job 11 completed at time 79.4865172122373
job 13 started at time 79.56428525937375
job 12 c